# Algorithm

In [1]:
import pandas as pd
from DBhelper import db_helper
import random
import numpy as np
import math

## Connection to database

In [2]:
db = db_helper(db='test_ticker')

### Step 1
- Define a time range
- Get valid stocks for that range

In [3]:
def get_ticker_sizes(db, start, end):
    ticker_sizes = []
    for ticker in db.get_tables():
        try:
            size = db.count_rows(ticker, start, end)
            ticker_sizes.append([ticker, size])
        except Exception as e:
            pass
    return ticker_sizes

        
def max_value(ticker_sizes):
    max_val = 0
    for ticker, size in ticker_sizes:
        if size > max_val:
            max_val = size
    return max_val
        

def valid_tickers(db, start, end):
    valid_tickers = []
    # Get a list of all tickers and its size
    ticker_sizes = get_ticker_sizes(db, start, end)
    # Get the max lenght for previous list
    max_val = max_value(ticker_sizes)
    # Store tickers which match max lenght
    for ticker, size in ticker_sizes:
        if size == max_val:
            valid_tickers.append(ticker)
    return valid_tickers

In [4]:
start = '2015-01-01'
end = '2017-01-01'

valid_tickers = valid_tickers(db, start, end)

len(valid_tickers)

2548

In [6]:
# Create a dictionary and store in memory data from DB for every ticker
# This way code will run faster
data = {}

for ticker in valid_tickers:
    df = db.get_data_between_dates(ticker, start, end)
    data[ticker] = df['Adj Close'].to_numpy()

### Step 2
- Define population size
- Create a population of 'n' individuals
- Asign a weight for each individual

In [23]:
class Individual:
    def __init__(self, ticker, db, start, end):
        self.ticker = ticker
        self.weight = np.random.random()
        self.db = db
        self.start = start
        self.end = end
        self.data = self.get_data()
        
    def get_data(self):
        return data[self.ticker]
#         df = self.db.get_data_between_dates(self.ticker, start, end)
#         return df['Adj Close'].to_numpy()
            
    def var(self):
        return np.var(self.data)
    
    def return_(self):
        return (self.data[-1] - self.data[0]) / self.data[0]
    
    def w_return(self):
        return self.return_() * self.weight
        
    def __eq__(self, obj):
        return self.ticker == obj.ticker
    
    def __str__(self):
        return f'{self.ticker} %.2f' % self.weight
    
    
class Portfolio:
    def __init__(self, size, tickers, db, start, end):
        self.size = size
        self.tickers = tickers
        self.new_portfolio(db, start, end)
        self.normalize_portfolio()
        
    def new_portfolio(self, db, start, end):
        portfolio = []
        while len(portfolio) < self.size:
            individual = Individual(random.choice(valid_tickers), db, start, end)
            if individual not in portfolio:
                portfolio.append(individual)
        self.portfolio = portfolio
        
    def normalize_portfolio(self):
        total_weight = sum([item.weight for item in self.portfolio])
        for item in self.portfolio:
            item.weight /= total_weight
    
    def get_cov(self, item1, item2):
        return np.cov(item1.data, item2.data)[0][1]
    
    def total_risk(self):
        sum1 = 0
        for item in self.portfolio:
            sum1 += item.weight * item.var()
        sum2 = 0
        for i in range(len(self.portfolio)-1):
            for j in range(i+1, len(self.portfolio)):
                item1, item2 = self.portfolio[i], self.portfolio[j]
                covar = self.get_cov(item1, item2)
                sum2 = item1.weight * item2.weight * covar
        return sum1 + sum2
       
    def total_return(self):
        return sum([item.w_return() for item in self.portfolio])
    
    def fitness(self):
        return self.total_return() - math.sqrt(self.total_risk())
        
    def __str__(self):        
        return ' , '.join([str(item) for item in self.portfolio])

In [29]:
PORTFOLIO_SIZE = 7
POPULATION_SIZE = 200

population = []
for _ in range(POPULATION_SIZE):
    population.append(Portfolio(PORTFOLIO_SIZE, valid_tickers, db, start, end))
    
print(population[0].fitness())

-12.286097420853853


## Step 3
- Select best individuals in population
- Crossover until population reach previous size
- Mutate genes with certain probability

In [30]:
population.sort(key=lambda x: x.fitness(), reverse=True)

In [32]:
best = population[0]
best.total_return(), best.total_risk()

(0.20677265218916016, 4.665115078987793)

In [35]:
def select_parents(population):
    weight_list = [portfolio.fitness() for portfolio in population]
    return random.choices(population, weights=weight_list, k=2)

# SOME LOGIC NEEDS TO BE IMPLEMENTED HERE!!!!
def crossover(parent1, parent2):
    child1 = parent1
    child2 = parent2
    return child1, child2

In [37]:
survivals = population[:int(POPULATION_SIZE/2)]

while len(survivals) <= POPULATION_SIZE:
    parent1, parent2 = select_parents(population)
    child1, child2 = crossover(parent1, parent2)
    survivals.append(child1)
    survivals.append(child2)
    
population = survivals

In [38]:
population.sort(key=lambda x: x.fitness(), reverse=True)

In [39]:
# TESTING, since no crossover logic has been implemented,
# best individual should match with previous one
best = population[0]
best.total_return(), best.total_risk()

(0.20677265218916016, 4.665115078987793)